<a href="https://colab.research.google.com/github/DevashishDeshmukh/AI_Lab_Assignments/blob/main/Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.stats import norm, poisson

# Define the population distribution by age group
ageGrpDistribution = np.array([0.38, 0.33, 0.23, 0.06])
pop_total = 8700000  # Total population of the city

# Define the initial number of normal and covid beds
MaxNormalBeds = 650
MaxCovidBeds = 0
discount_factor=0.9
# Define the cost parameters
DOS_cost_normal_beds = 500
DOS_cost_covid_beds = 2000
converted_beds = 10000

# Define the range of beds to convert from normal to covid
beds_convert_range = np.arange(0, MaxNormalBeds + 1, 50)

# Define the transition function
def transition_prob_func(state, action):
    # Unpack the state
    ageGrpDistribution, pop_total, beds_normal, beds_covid = state
    
    # Calculate the number of requests for normal and covid beds
    requests_normal = np.sum(poisson.rvs(0.1, size=len(ageGrpDistribution)) * ageGrpDistribution)
    requests_covid = np.sum(poisson.rvs(0.05, size=len(ageGrpDistribution)) * ageGrpDistribution)
    
    # Calculate the number of discharges for normal and covid beds
    discharges_normal = poisson.rvs(beds_normal)
    discharges_covid = poisson.rvs(beds_covid,)
    
    # Calculate the number of denied requests for normal and covid beds
    denied_normal = max(requests_normal - beds_normal - action, 0)
    denied_covid = max(requests_covid - beds_covid, 0)
    
    # Calculate the new state after accounting for the action and the probabilistic transitions
    newNormalBeds = max(min(beds_normal + requests_normal - discharges_normal - action, MaxNormalBeds), 0)
    newCovidBeds = max(min(beds_covid + requests_covid - discharges_covid, MaxNormalBeds), 0)
    new_ageGrpDistribution = ageGrpDistribution + norm.rvs(scale=1, size=len(ageGrpDistribution))
    newTotalPop = max(min(pop_total + norm.rvs(scale=0.1), 10000000), 0)
    
    # Calculate the reward for the transition
    reward = -DOS_cost_normal_beds * denied_normal \
             -DOS_cost_covid_beds * denied_covid \
             -converted_beds * action
    
    return ((new_ageGrpDistribution, newTotalPop, newNormalBeds, newCovidBeds), reward)

# Define the value iteration function to solve the MDP
def value_iteration(theta=0.0001, discount_factor=0.9):
    V = np.zeros((ageGrpDistribution.size+2, MaxNormalBeds+1, 4))
    delta = float('inf')
    while delta > theta:
        delta = 0
        for age_group in range(ageGrpDistribution.size+2):
            for beds_normal in range(MaxNormalBeds+1):
                for week in range(4):
                    v = V[age_group, beds_normal, week]
                    action_values = []
                    for action in beds_convert_range:
                        next_state, reward = transition_prob_func((ageGrpDistribution, pop_total, beds_normal, MaxCovidBeds), action)
                        next_age_group = np.clip(np.sum(next_state[0] * ageGrpDistribution / next_state[1]), 0                        , len(ageGrpDistribution)+1)
                        next_beds_normal = next_state[2]
                        next_week = week + 1 if week < 3 else 0
                        action_values.append(reward + discount_factor * V[next_age_group, next_beds_normal, next_week])
                    V[age_group, beds_normal, week] = max(action_values)
                    delta = max(delta, abs(v - V[age_group, beds_normal, week]))
    return V

# Solve the MDP
V = value_iteration()

# Extract the optimal policy
policy = np.zeros((ageGrpDistribution.size+2, MaxNormalBeds+1, 4))
for age_group in range(ageGrpDistribution.size+2):
    for beds_normal in range(MaxNormalBeds+1):
        for week in range(4):
            action_values = []
            for action in beds_convert_range:
                next_state, reward = transition_prob_func((ageGrpDistribution, pop_total, beds_normal, MaxCovidBeds), action)
                next_age_group = np.clip(np.sum(next_state[0] * ageGrpDistribution / next_state[1]), 0, len(ageGrpDistribution)+1)
                next_beds_normal = next_state[2]
                next_week = week + 1 if week < 3 else 0
                action_values.append(reward + discount_factor * V[next_age_group, next_beds_normal, next_week])
            policy[age_group, beds_normal, week] = beds_convert_range[np.argmax(action_values)]

# Print the optimal policy for each week
for week in range(4):
    print(f"Week {week+1}:")
    print("Age Group\tNormal Beds\tConvert to Covid Beds")
    for age_group in range(ageGrpDistribution.size+2):
        for beds_normal in range(MaxNormalBeds+1):
            print(f"{age_group}\t\t{beds_normal}\t\t{policy[age_group, beds_normal, week]}")



IndexError: ignored